In [4]:
from code_data_science import data_table as dt
from code_data_science import palette
import plotly.express as px
import plotly.graph_objects as go
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
import numpy as np
from infinity_emb import EngineArgs, AsyncEmbeddingEngine
import asyncio
import time
import logging

# Adjust the log level for the 'infinity_emb' package to WARNING or higher
logging.getLogger("infinity_emb").setLevel(logging.ERROR)

# Get data
df = dt.read_csv("../samples/find_methods.csv")
df.drop_duplicates("method", inplace=True)

# Initialize the engine outside the function
engine_args = EngineArgs(
    model_name_or_path="michaelfeil/bge-small-en-v1.5",
    device="cpu",
    engine="optimum",
    compile=True,
    batch_size=2
)
engine = AsyncEmbeddingEngine.from_args(engine_args)

# Define a single function to get embeddings
async def get_embeddings(sentences: list[str]) -> list:
    async with engine:  # Ensure engine is properly started and stopped
        embeddings, _ = await engine.embed(sentences=sentences)
    return embeddings

# Track time for embeddings
start_time_embeddings = time.time_ns()
loop = asyncio.get_running_loop()
embeddings = await loop.create_task(get_embeddings(df["method"].to_list()))
end_time_embeddings = time.time_ns()
embedding_time_seconds = (end_time_embeddings - start_time_embeddings) / 1e9

embeddings = np.array(embeddings)
# Track time for t-SNE
start_time_tsne = time.time_ns()
indices = TSNE(n_components=2, perplexity=30, random_state=42).fit_transform(embeddings)
end_time_tsne = time.time_ns()
tsne_time_seconds = (end_time_tsne - start_time_tsne) / 1e9

x = indices[:, 0]
y = indices[:, 1]

# Apply Gaussian Mixture Model
start_time_gmm = time.time_ns()
gmm = GaussianMixture(n_components=6, covariance_type='full', random_state=42)
labels = gmm.fit_predict(embeddings)
end_time_gmm = time.time_ns()
gmm_time_seconds = (end_time_gmm - start_time_gmm) / 1e9

df["x"] = x
df["y"] = y
df["cluster id"] = labels.astype("str")  # As string so the colors and legend are discrete
df.sort_values(by="cluster id", inplace=True)

custom_palette = [
    palette.__moderne_color_map["red"]["main"],
    palette.__moderne_color_map["yellow"]["main"],
    palette.__moderne_color_map["green"]["main"],
    palette.__moderne_color_map["blue"]["main"],
    palette.__moderne_color_map["indigo"]["main"],
    palette.__moderne_color_map["red"][700],
    palette.__moderne_color_map["yellow"][700],
]

colors = custom_palette[:gmm.n_components]

# Create the figure with time metrics in the title
fig = px.scatter(
    df,
    x="x",
    y="y",
    log_x=False,
    color="cluster id",
    hover_name="method",
    color_discrete_sequence=colors
)

# Function to draw ellipses
def draw_ellipse_plotly(mean, covariance, fig, n_std=3, **kwargs):
    """Draw an ellipse with a given mean and covariance using Plotly."""
    # Convert covariance to principal axes
    U, s, Vt = np.linalg.svd(covariance)
    angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
    
    # Handle the width and height correctly based on the 2D covariance matrix
    width, height = 2 * np.sqrt(s[:2])  # Take the first two singular values
    
    # Generate ellipse coordinates
    theta = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = width / 2 * np.cos(theta)
    ellipse_y = height / 2 * np.sin(theta)

    # Rotation matrix based on the angle
    R = np.array([
        [np.cos(np.radians(angle)), -np.sin(np.radians(angle))],
        [np.sin(np.radians(angle)), np.cos(np.radians(angle))]
    ])
    
    # Rotate the ellipse coordinates
    ellipse_coords = np.dot(R, np.array([ellipse_x, ellipse_y]))
    
    # Add ellipses to the plot for each standard deviation level
    for nsig in range(1, n_std + 1):
        fig.add_trace(
            go.Scatter(
                x=ellipse_coords[0] * nsig + mean[0],
                y=ellipse_coords[1] * nsig + mean[1],
                mode='lines',
                **kwargs
            )
        )

# Draw ellipses around GMM clusters
for mean, covar in zip(gmm.means_, gmm.covariances_):
    draw_ellipse_plotly(mean, covar, fig, n_std=3, line=dict(color='red'))

fig.update_layout(
    showlegend=False,
    title=(
        f"Embedding Time: {embedding_time_seconds:.2f} sec | "
        f"t-SNE Time: {tsne_time_seconds:.2f} sec | "
        f"GMM Time: {gmm_time_seconds:.2f} sec"
    )
)

fig.show()

The ONNX file model_quantized_optimized.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.
/Users/juju/miniconda3/envs/moderne/lib/python3.10/site-packages/plotly/express/_core.py:1992: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

